In [187]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [188]:
athletesDf = pd.read_csv('../originalDataSet/athlete_events.csv', encoding="ISO-8859-1") # Dataset original
regions = pd.read_csv('../originalDataSet/noc_regions.csv', encoding="ISO-8859-1") # Regiões do dataset original
#countryContinent = pd.read_csv('../originalDataSet/countryContinent.csv', encoding="ISO-8859-1") # Csv com países com continente e regiões
NOCCountryContinent = pd.read_csv('../cleanedDataSet/NOC_Country_Continent_Cleaned.csv', encoding="ISO-8859-1") # Dados tratados com NOC original e seu continente e região
yearCountryHost = pd.read_csv('../originalDataSet/yearCountryHost.csv') # Csv com ano e país que aconteceu a olimpiada
summerYearCountry = yearCountryHost[~yearCountryHost['Summer'].isna()]
winterYearCountry = yearCountryHost[~yearCountryHost['Winter'].isna()]

athletesDf = athletesDf[athletesDf['Year'].isin(range(1980,2040))]

summerAthletes = athletesDf[athletesDf['Season'].isin(['Summer'])]
winterAthletes = athletesDf[athletesDf['Season'].isin(['Winter'])]

summerOlimpWithHostCountry = pd.merge(summerAthletes, summerYearCountry[["Year","Country"]], how = "left", on = ["Year"])
winterOlimpWithHostCountry = pd.merge(winterAthletes, winterYearCountry[["Year","Country"]], how = "left", on = ["Year"])
summerOlimpWithHostCountry.rename(columns = {'Country':'HostCountry'}, inplace = True)
winterOlimpWithHostCountry.rename(columns = {'Country':'HostCountry'}, inplace = True)
summerAthletesCompleteRegions = pd.merge(summerOlimpWithHostCountry, NOCCountryContinent[["NOC", "continent", "sub_region"]], how = "left",on = ["NOC"])
winterAthletesCompleteRegions = pd.merge(winterOlimpWithHostCountry, NOCCountryContinent[["NOC", "continent", "sub_region"]], how = "left",on = ["NOC"])
summerAthletesWithMedals = summerAthletesCompleteRegions[~summerAthletesCompleteRegions['Medal'].isna()]
winterAthletesWithMedals = winterAthletesCompleteRegions[~winterAthletesCompleteRegions['Medal'].isna()]

summerBoxJudoTaekwondoWrestling = summerAthletesWithMedals[summerAthletesWithMedals['Sport'].isin(['Judo', 'Taekwondo', 'Boxing', 'Wrestling'])]
summerAthletesWithMedals = summerAthletesWithMedals[~summerAthletesWithMedals['Sport'].isin(['Judo', 'Taekwondo', 'Boxing', 'Wrestling'])]
dfSummerMedalsAndRegions = summerAthletesWithMedals.drop_duplicates(subset = ["Event","Medal"],keep = "first")
dfSummerMedalsAndRegions = pd.concat([dfSummerMedalsAndRegions, summerBoxJudoTaekwondoWrestling], axis=0)

summerCount = dfSummerMedalsAndRegions.groupby(['Year', 'sub_region']).agg('count').reset_index()

meanMedalsSubContinent = summerCount.groupby(['sub_region'])['Medal'].agg('mean').reset_index() 
fig = px.histogram(meanMedalsSubContinent, x="sub_region", y="Medal", histnorm='percent')
fig.show()

summerCount = dfSummerMedalsAndRegions.groupby(['Year', 'continent']).agg('count').reset_index()
meanMedalsContinent = summerCount.groupby(['continent'])['Medal'].agg('mean').reset_index()

summerComparison = dfSummerMedalsAndRegions.groupby(['Year'])
summerComparison = summerComparison.get_group(2008).groupby(['continent']).agg('count').reset_index()
summerComparison.rename(columns = {'Medal':'MedalComparison'}, inplace = True)

# Calculate Percentage
meanMedalsContinent['medal_percent'] = (meanMedalsContinent['Medal'] / 
                      meanMedalsContinent['Medal'].sum()) * 100

summerComparison['MedalComparisonPercent'] = (summerComparison['MedalComparison'] / 
                      summerComparison['MedalComparison'].sum()) * 100

orgAndComparison = pd.merge(meanMedalsContinent, summerComparison[["continent","MedalComparisonPercent"]], how = "left", on = ["continent"])
#print(orgAndComparison)
fig = px.bar(meanMedalsContinent, x="medal_percent", y="continent", orientation='h')
fig = px.bar(orgAndComparison, x=["medal_percent", "MedalComparisonPercent"], y="continent", orientation='h', barmode = 'group')
fig.update_layout(yaxis={'categoryorder':'total ascending'})


#fig = px.bar(summerComparison, x="medal_percent", y="continent", orientation='h')
#fig.update_layout(yaxis={'categoryorder':'total ascending'})

#fig = px.histogram(meanMedalsContinent, x="continent", y="Medal", histnorm='percent')
#fig = px.histogram(meanMedalsContinent, x="continent", y="Medal", histnorm='percent')
#fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


#athletesWithMedal = athletesDf[~athletesDf['Medal'].isna()]
#athletesWithGoldMedal = athletesWithMedal[athletesWithMedal['Medal'].isin(['Gold'])]
#swimAthletesWithGoldMedal = athletesWithGoldMedal[athletesWithGoldMedal['Sport'].isin(['Swimming'])]
#swimMaleAthletesWithGoldMedal = athletesWithGoldMedal[athletesWithGoldMedal['Sex'].isin(['M'])]
#swimMaleAthletesWithGoldMedal = swimMaleAthletesWithGoldMedal.groupby('Year')
#meanWeightPerYear = swimMaleAthletesWithGoldMedal['Weight'].mean()

#plt.plot(meanWeightPerYear, label="Média geral")
#outer_merged.head()
#plt.plot(meanWeightPerYear.index.get_level_values('Year').unique())
#plt.legend()
#plt.set_ylabel("Altura Média (cm)")
#plt.set_title("Altura média de homens e mulheres ao longo dos jogos")
